In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
The dataset looks like:

	       Timestamp	CAN ID	DLC	X0	X1	X2	X3	X4	X5	X6	X7	Flag	Type
0	     1.478198e+09	0316	8	05	21	68	09	21	21	00	6f	R	Normal
1	     1.478198e+09	018f	8	fe	5b	00	00	00	3c	00	00	R	Normal
2	     1.478198e+09	0260	8	19	21	22	30	08	8e	6d	3a	R	Normal
3	     1.478198e+09	02a0	8	64	00	9a	1d	97	02	bd	00	R	Normal
4	     1.478198e+09	0329	8	40	bb	7f	14	11	20	00	14	R	Normal
...	...	...	...	...	...	...	...	...	...	...	...	...	...
3665766	 1.478201e+09	018f	8	fe	59	00	00	00	41	00	00	R	Normal
3665767	 1.478201e+09	0260	8	18	21	21	30	08	8f	6d	19	R	Normal
3665768	 1.478201e+09	02a0	8	24	00	9a	1d	97	02	bd	00	R	Normal
3665769	 1.478201e+09	0329	8	dc	b7	7f	14	11	20	00	14	R	Normal
3665770	 1.478201e+09	0545	8	d8	00	00	8b	00	00	00	00	R	Normal

This is the dataset for one of the four type of attacks.
There are similar other three.

In [ ]:
# for using the drive we first mounted it.The dataset was downloaded from intenet
DoS = pd.read_csv('/content/drive/MyDrive/DoS_dataset.csv', names = ['Timestamp','CAN ID', 'DLC', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Flag'])
DoS['Type'] = 'Normal'
DoS.loc[DoS["Flag"] == "T", "Type"] = 'DoS'
DoS = DoS.dropna()

In [ ]:
Fuzzy = pd.read_csv('/content/drive/MyDrive/Fuzzy_dataset.csv', names = ['Timestamp','CAN ID', 'DLC', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Flag'])
Fuzzy['Type'] = 'Normal'
Fuzzy.loc[Fuzzy["Flag"] == "T", "Type"] = 'Fuzzy'
Fuzzy = Fuzzy.dropna()

In [ ]:
RPM = pd.read_csv('/content/drive/MyDrive/RPM_dataset.csv', names = ['Timestamp','CAN ID', 'DLC', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Flag'])
RPM['Type'] = 'Normal'
RPM.loc[RPM["Flag"] == "T", "Type"] = 'RPM'
RPM = RPM.dropna()

In [ ]:
gear = pd.read_csv('/content/drive/MyDrive/gear_dataset.csv', names = ['Timestamp','CAN ID', 'DLC', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Flag'])
gear['Type'] = 'Normal'
gear.loc[gear["Flag"] == "T", "Type"] = 'gear'
gear = gear.dropna()

In [ ]:
#concatenating all the 4 dataframes into one dataframe
frames = [DoS, Fuzzy, RPM, gear]
df = pd.concat(frames, ignore_index=True)

In [ ]:
#label encoding
df['Type'] = df['Type'].map({'Normal':0, 'DoS':1, 'Fuzzy':2, 'RPM':3, 'gear':4})

#Sampling 2 million data rows from concatenated dataset
df_used, df_unused = train_test_split(df, train_size=0.125, stratify=df['Type'])
df['Type'].unique()

In [ ]:
#Saving the dataset for further use
df_used.to_csv('/content/drive/MyDrive/final_df.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/final_df.csv')
df = df.drop(columns = ['Unnamed: 0'], axis = 0)

In [ ]:
# Convert the timestamp column to a datetime object
df.head(500)['Timestamp'] = pd.to_datetime(df.head(500)['Timestamp'], unit='s')

# Create a separate plot for each X column
fig, ax = plt.subplots(figsize=(9,6))
colors = ['red', 'blue', 'green', 'orange','black']

for i, typ in enumerate(df.head(500)['Type'].unique()):
    df_type = df.head(500)[df.head(500)['Type']==typ]
    ax.scatter(df_type['Timestamp'], df_type['Type'], label=typ, alpha=0.6, s=40, color=colors[i % len(colors)])

ax.set_title('Type over time')
ax.set_xlabel('Timestamp')
ax.set_ylabel('Type')
ax.legend(loc='upper left')
plt.style.use('ggplot')
plt.show()

In [ ]:
#Dropping columns
df = df.drop(['Timestamp', 'CAN ID', 'DLC', 'Flag'], axis=1)

#Defining feature set and output set
X = df.drop(['Type'], axis=1)
y = df['Type']

In [ ]:
#Label encoding the byte data
for i in X:
  le = sklearn.preprocessing.LabelEncoder()
  X[i] = le.fit_transform(X[i])

In [ ]:
The dataset after processing looks like:


        X0	 X1	 X2	 X3	 X4	 X5	 X6	 X7
0	 0	 0	 0	 0	 0	 4	 7	 116
1	 0	 32	 0	 0	 0	 0	 0	 0
2	 128     138     0	 0	 34	 0	 0	 0
3	 5	 32	 180     104     113     0	 0	 136
4	 0	 0	 0	 0	 0	 8	 4	 36
...	 ...	...	...	...	...	...	...	...
2046096	 18	 128     0	 255     249     127     5	 229
2046097	 1	 69	 96	 255     107     0	 0	 0
2046098	 247     127     0	 0	 46	 127     13	 180
2046099	 0	 0	 0	 128     0	 104     209     19
2046100	 5	 32	 52	 104     117     0	 0	 12

In [ ]:
#Splitting the data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
#Label encoding the df
for i in df:
  le = sklearn.preprocessing.LabelEncoder()
  df[i] = le.fit_transform(df[i])

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
The models start from here.

In [ ]:
#Instantiate DTC model
dtc = DecisionTreeClassifier(max_depth=11)

#Fit the model
dtc.fit(X_train, y_train)

#Predict
dtc_pred = dtc.predict(X_test)
dtc_score = accuracy_score(y_test, dtc_pred)

print("Decision tree classifier accuracy is", dtc_score)
print(classification_report(y_test, dtc_pred))
#plot the decision tree
fn = ['X0','X1','X2','X3','X4','X5','X6','X7']
cn = ['Normal', 'DoS', 'Fuzzy', 'RPM', 'gear']
fig = plt.figure(figsize=(160,90))
tree.plot_tree(dtc, feature_names=fn, class_names = cn, filled=True)

In [ ]:
rfc=RandomForestClassifier(random_state=42,n_estimators=20,max_depth=11)
rfc.fit(X_train, y_train)
rtf_y_pred = rfc.predict(X_test)
rtf_score = accuracy_score(y_test, rtf_y_pred)
print("Accuracy of optimized random forest classifier", rtf_score)
print("Classification report - random forest classifier",classification_report(y_test, rtf_y_pred))

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y = clf.predict(X_test)
gnb_score = accuracy_score(y,y_test)
print("Accuracy of base Gaussian NB Classifier:", gnb_score)
print("Classification Report - Gaussian NB Classifier:", classification_report(y,y_test))

In [ ]:
clf = AdaBoostClassifier(n_estimators=20, random_state=0)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
adb_score = accuracy_score(y_test, pred)
print("Accuracy score of AdaBoost Classifier is",adb_score)
print("Classification Report - AdaBoost Classifier is",classification_report(y_test, pred))

In [ ]:
# Comparative visualisation of the model accuracies
model_scores = [dtc_score, rtf_score, gnb_score, adb_score]
x_axis = ['DecisionTree', 'RandomForest', 'Gaussian', 'AdaBoost']
plt.plot(x_axis, model_scores)
plt.title('Model Scores')
plt.xlabel("Models")
plt.ylabel("Accuracy")